In [1]:
from dotenv import load_dotenv
import os

In [2]:
load_dotenv() 
API_KEY = os.getenv("GEMINI_API_KEY")

In [3]:
from google import genai

In [4]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [8]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [9]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [13]:
def llm(prompt):
    client = genai.Client(api_key=API_KEY)
    responses = client.models.generate_content(
        model="gemini-2.0-flash", contents=prompt)
    return responses.text

In [15]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [16]:
rag('how do I run kafka?')

'To run Kafka, refer to the following options:\n\n1.  For Java Kafka, in the project directory, run: `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`.\n2.  For Python Kafka, create a virtual environment, activate it, and install the required packages from `requirements.txt`. You can create and activate the virtual environment by running:\n    ```\n    python -m venv env\n    source env/bin/activate\n    pip install -r ../requirements.txt\n    ```\n    Remember to activate the virtual environment each time you need it using `source env/bin/activate` (use `env/Scripts/activate` on Windows). You can deactivate it using `deactivate`. Also use `pip install kafka-python-ng` instead of the usual installation.\n3.  If you encounter a "Permission denied" error while running `./build.sh`, execute `chmod +x build.sh` in the same directory (/docker/spark).\n'

In [17]:
rag('the course has already started, can I still enroll?')

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute.\n"


RAG with Vector Search


In [18]:
from qdrant_client import QdrantClient, models

In [19]:
qd_client = QdrantClient("http://localhost:6333")

In [20]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [21]:
collection_name = "zoomcamp-faq"

In [22]:
qd_client.delete_collection(collection_name = collection_name)

False

In [23]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [26]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [27]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [28]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

In [29]:
question = 'I just discovered the course. Can I still join it?'

In [30]:
def vector_search(question):
    print('vector_search is used')
    
    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [31]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [32]:
rag('how do I run kafka?')

vector_search is used


"To run Kafka, you can refer to the following methods depending on your specific needs:\n\n*   **For Java Kafka producer/consumer/kstreams:** In the project directory, run `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`.\n*   **For Python producer:** Create a virtual environment, activate it, and install the requirements from `requirements.txt`. Then run the Python file within that environment.\n*   **If you encounter a `kafka.errors.NoBrokersAvailable` error:** Ensure your Kafka broker Docker container is running by using `docker ps` to confirm. If it's not running, navigate to the Docker Compose YAML file folder and run `docker compose up -d` to start all instances."